In [ ]:
import os
import datetime
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
(train_data,test_data),df_info = tfds.load('food101',
                                           split=['train','validation'], # in our case we have training and validation but some dataset have testing too
                                           shuffle_files=True, # we are shuffle our files to some randomness in our data
                                           as_supervised=True, # beacuse our dataset is supervised
                                           with_info=True) # getting metadata about our dataset

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/75750 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteWI99YW/food101-train.tfrecord*...:   0%|          …

Generating validation examples...:   0%|          | 0/25250 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/food101/2.0.0.incompleteWI99YW/food101-validation.tfrecord*...:   0%|     …

Dataset food101 downloaded and prepared to /root/tensorflow_datasets/food101/2.0.0. Subsequent calls will reuse this data.


In [ ]:
def preprocess_image_data(images,labels,img_size=(224,224)):
  image  = tf.image.resize(images,img_size)
  return tf.cast(image,dtype=tf.float32),labels

In [ ]:
train_data = train_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)
test_data  = test_data.map(map_func=preprocess_image_data,num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def create_tensorboard_callback(dir_name,experiment):
  date_time = datetime.datetime.now().strftime('%Y/%m/%d:%H-%M-%S')
  path = os.path.join(dir_name,experiment,date_time)
  return tf.keras.callbacks.TensorBoard(log_dir=path)

checkpoint_path = 'amelialwx/model_checkpoints/cp.cpkt'

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         monitor='val_accuracy',
                                                         save_best_only=True,
                                                         save_weights_only=True,
                                                         verbose=0)

mixed_precision.set_global_policy('mixed_float16')
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-d583988e-c92a-4039-bf9e-e27f45cbfcaa)


In [ ]:
mixed_precision.global_policy()

<Policy "mixed_float16">

In [ ]:
input_shape = (224,224,3)

base_model = tf.keras.applications.EfficientNetV2B0(include_top=False)
base_model.trainable = False

input = layers.Input(shape=input_shape,name='input_layer')
x = base_model(input)
x = layers.GlobalAveragePooling2D(name='globalAvgPooling_layer')(x)
x = layers.Dense(101,name='output_layer')(x)
output = layers.Activation(activation='softmax',dtype=tf.float32,name='prediction')(x)
model = tf.keras.Model(input,output,name='food_vision_model')
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.summary()

24274472/24274472 [==============================] - 0s 0us/step
Model: "food_vision_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetv2-b0 (Functio  (None, None, None, 1280   5919312   
 nal)                        )                                   
                                                                 
 globalAvgPooling_layer (Gl  (None, 1280)              0         
 obalAveragePooling2D)                                           
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
 prediction (Activation)     (None, 101)               0         
                                                  

In [ ]:
feature_extraction_history = model.fit(train_data,
                                       epochs=5,
                                       steps_per_epoch=len(train_data),
                                       validation_data=test_data,
                                       validation_steps=int(.15*len(test_data)),
                                       callbacks=[create_tensorboard_callback('models','FX_efficientnet0'),checkpoint_callback])

Epoch 1/5
2368/2368 [==============================] - 211s 77ms/step - loss: 1.8396 - accuracy: 0.5547 - val_loss: 1.1772 - val_accuracy: 0.6928
Epoch 2/5
2368/2368 [==============================] - 185s 77ms/step - loss: 1.3177 - accuracy: 0.6603 - val_loss: 1.0471 - val_accuracy: 0.7238
Epoch 3/5
2368/2368 [==============================] - 185s 77ms/step - loss: 1.1830 - accuracy: 0.6910 - val_loss: 0.9878 - val_accuracy: 0.7344
Epoch 4/5
2368/2368 [==============================] - 181s 75ms/step - loss: 1.1039 - accuracy: 0.7095 - val_loss: 0.9731 - val_accuracy: 0.7394
Epoch 5/5
2368/2368 [==============================] - 180s 75ms/step - loss: 1.0404 - accuracy: 0.7239 - val_loss: 0.9646 - val_accuracy: 0.7399


In [ ]:
base_model.trainable = True

for layer in base_model.layers[:-20]:
  layer.trainable = False

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                                  patience=3)

checkpoint_path = "amelialwx/fine_tune_checkpoints/"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      save_best_only=True,
                                                      monitor="val_loss")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                                 factor=0.2,
                                                 patience=2,
                                                 verbose=1,
                                                 min_lr=1e-7)
best_model = model.fit(train_data,
                       epochs=100,
                       steps_per_epoch=len(train_data),
                       validation_data=test_data,
                       validation_steps=int(.15 * len(test_data)),
                       callbacks=[create_tensorboard_callback('models','best_fine_effb0'),
                                  early_stopping,reduce_lr,model_checkpoint])

Epoch 1/100
2368/2368 [==============================] - 241s 93ms/step - loss: 1.0213 - accuracy: 0.7334 - val_loss: 0.8771 - val_accuracy: 0.7630 - lr: 1.0000e-04
Epoch 2/100
2368/2368 [==============================] - 222s 93ms/step - loss: 0.8700 - accuracy: 0.7683 - val_loss: 0.8255 - val_accuracy: 0.7765 - lr: 1.0000e-04
Epoch 3/100
2368/2368 [==============================] - 223s 92ms/step - loss: 0.7890 - accuracy: 0.7888 - val_loss: 0.8116 - val_accuracy: 0.7794 - lr: 1.0000e-04
Epoch 4/100
2368/2368 [==============================] - 222s 93ms/step - loss: 0.7245 - accuracy: 0.8067 - val_loss: 0.7810 - val_accuracy: 0.7871 - lr: 1.0000e-04
Epoch 5/100
2368/2368 [==============================] - 237s 99ms/step - loss: 0.6721 - accuracy: 0.8206 - val_loss: 0.7678 - val_accuracy: 0.7948 - lr: 1.0000e-04
Epoch 6/100
2368/2368 [==============================] - 229s 96ms/step - loss: 0.6233 - accuracy: 0.8331 - val_loss: 0.7619 - val_accuracy: 0.7961 - lr: 1.0000e-04
Epoch 7/10

In [ ]:
def preprocess_image(bytes_input):
    decoded = tf.io.decode_jpeg(bytes_input, channels=3)
    decoded = tf.image.convert_image_dtype(decoded, tf.float32)
    resized = tf.image.resize(decoded, size=(224, 224))
    return resized

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def preprocess_function(bytes_inputs):
    decoded_images = tf.map_fn(
        preprocess_image, bytes_inputs, dtype=tf.float32, back_prop=False
    )
    return {"model_input": decoded_images}

@tf.function(input_signature=[tf.TensorSpec([None], tf.string)])
def serving_function(bytes_inputs):
    images = preprocess_function(bytes_inputs)
    results = model_call(**images)
    return results

model_call = tf.function(model.call).get_concrete_function(
    [
        tf.TensorSpec(
            shape=[None, 224, 224, 3], dtype=tf.float32, name="model_input"
        )
    ]
)

In [ ]:
import os
os.makedirs('/content/amelialwx/with_signature', exist_ok=True)

In [ ]:
tf.saved_model.save(
    model,
    '/content/amelialwx/with_signature',
    signatures={"serving_default": serving_function},
)

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
Instructions for updating:
Use fn_output_signature instead


In [ ]:
model.save('/content/amelialwx/model')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

img = image.load_img('/content/amelialwx/image.jpg')
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = preprocess_input(img_array)
predictions = model.predict(img_array)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
import cv2
from collections import Counter

img = image.load_img('/content/amelialwx/image.jpg')
img_array = image.img_to_array(img)

def center_crop(x, center_crop_size):
    centerw, centerh = x.shape[0]//2, x.shape[1]//2
    halfw, halfh = center_crop_size[0]//2, center_crop_size[1]//2
    return x[centerw-halfw:centerw+halfw, centerh-halfh:centerh+halfh, :]

def predict_10_crop(img, ix, top_n=5, plot=False, preprocess=True, debug=False):
    crop_size = (224, 224)
    flipped_X = np.fliplr(img)

    crops = [
        img[:crop_size[0], :crop_size[1], :], # Upper Left
        img[:crop_size[0], -crop_size[1]:, :], # Upper Right
        img[-crop_size[0]:, :crop_size[1], :], # Lower Left
        img[-crop_size[0]:, -crop_size[1]:, :], # Lower Right
        center_crop(img, crop_size), # Center
        flipped_X[:crop_size[0], :crop_size[1], :], # Flipped Upper Left
        flipped_X[:crop_size[0], -crop_size[1]:, :], # Flipped Upper Right
        flipped_X[-crop_size[0]:, :crop_size[1], :], # Flipped Lower Left
        flipped_X[-crop_size[0]:, -crop_size[1]:, :], # Flipped Lower Right
        center_crop(flipped_X, crop_size) # Flipped Center
    ]

    crops = [cv2.resize(x, (224, 224)) for x in crops]
    if plot:
        fig, axes = plt.subplots(2, 5, figsize=(10, 4))
        for i, ax in enumerate(axes.flat):
            ax.imshow(crops[i].astype('uint8'))
            ax.axis('off')
        plt.show()

    y_pred = model.predict(np.stack(crops, axis=0))
    preds = np.argmax(y_pred, axis=1)
    top_n_preds = np.argpartition(y_pred, -top_n)[:,-top_n:]

    if debug:
        print('Top-1 Predicted:', preds)
        print('Top-5 Predicted:', top_n_preds)

    return preds, top_n_preds

preds, top_n_preds = predict_10_crop(img_array, ix=0, preprocess=False, debug=True)
most_common_pred, count = Counter(preds).most_common(1)[0]
most_common_pred

1/1 [==============================] - 0s 63ms/step
Top-1 Predicted: [76 76 76 76 76 76 76 76 76 76]
Top-5 Predicted: [[47  8  0 67 76]
 [67 66 76 59 82]
 [18  0 76 67 47]
 [49 46 59 76  0]
 [ 8  0 67 59 76]
 [67 82 76 59 66]
 [46  8 67  0 76]
 [ 0  8 59 46 76]
 [ 0 18 76 67 47]
 [47  0 67 59 76]]


76